In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.1 MB/s eta 0:00:00


In [ ]:
import os
import faiss
import torch
import numpy as np
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import kagglehub

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device}")

🚀 Using device: cuda


In [ ]:
import kagglehub
path = kagglehub.dataset_download("benhamner/nips-papers")
print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/papers.csv")
texts = df['paper_text'].fillna('')

100%|██████████| 141M/141M [00:08<00:00, 18.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/benhamner/nips-papers/versions/2


In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'\[[^\]]*\]', '', text)
    text = re.sub(r'\n', ' ', text)
    return text.strip()

df['clean_text'] = df['paper_text'].apply(clean_text)

In [ ]:
print("📊 Training LDA model...")
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['clean_text'])
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda_matrix = lda.fit_transform(tfidf_matrix)

📊 Training LDA model...


In [ ]:
def display_topics(model, feature_names, top_words=5):
    for idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-top_words - 1:-1]]
        print(f"🧠 Topic {idx+1}: {', '.join(top_features)}")

display_topics(lda, vectorizer.get_feature_names_out())

🧠 Topic 1: policy, regret, reward, action, state
🧠 Topic 2: tensor, pca, manifold, principal, rank
🧠 Topic 3: model, data, posterior, distribution, models
🧠 Topic 4: image, training, model, learning, images
🧠 Topic 5: algorithm, matrix, data, learning, theorem
🧠 Topic 6: visual, stimulus, cells, motion, model
🧠 Topic 7: topic, document, word, words, dirichlet
🧠 Topic 8: network, input, neural, neurons, networks
🧠 Topic 9: tree, node, nodes, trees, belief
🧠 Topic 10: ci, 32, kl, ii, ij


In [ ]:
print("🔗 Generating embeddings...")
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
embeddings = model.encode(df['clean_text'].tolist(), convert_to_numpy=True, normalize_embeddings=True, batch_size=32, show_progress_bar=True)

🔗 Generating embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/227 [00:00<?, ?it/s]

In [ ]:
print("🧠 Building FAISS index...")
dimension = embeddings.shape[1]
faiss.normalize_L2(embeddings)
index = faiss.IndexHNSWFlat(dimension, 32)
index.add(embeddings)

🧠 Building FAISS index...


In [ ]:
def retrieve_papers(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    faiss.normalize_L2(query_embedding)
    _, idxs = index.search(query_embedding, top_k)
    return df.iloc[idxs[0]][['title', 'paper_text']]

In [ ]:
print("📝 Loading summarization model...")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0 if device == "cuda" else -1)

📝 Loading summarization model...


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def summarize_text(text):
    max_chunk = 500
    text = clean_text(text)
    sentences = text.split('. ')
    current_chunk = ''
    chunks = []

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chunk:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '
    chunks.append(current_chunk.strip())

    summaries = []
    for chunk in chunks[:2]:  # summarize only top 2 chunks
        summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    return ' '.join(summaries)

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
def gradio_chatbot(query):
    papers = retrieve_papers(query)
    output = ""
    for i, (title, text) in enumerate(zip(papers['title'], papers['paper_text'])):
        summary = summarize_text(text)
        output += f"### 📖 {i+1}. {title}\n"
        output += f"🔹 {summary}\n\n"
    return output.strip()

# Gradio Interface
interface = gr.Interface(
    fn=gradio_chatbot,
    inputs=gr.Textbox(placeholder="Ask about Hidden Markov Models or any topic...", lines=2, label="🔍 Research Query"),
    outputs=gr.Markdown(label="📚 Results"),
    title="🧠 AI-Powered Research Paper Finder",
    description="Enter a query (e.g., 'hidden markov model', 'neural networks') to get relevant NIPS paper summaries.",
    theme="default",
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e2e0d46c6c5f6aeec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
if __name__ == "__main__":
    chatbot("Any paper about machine learning?")


🔍 Searching for relevant research...

📖 1. Using Machine Learning to Break Visual Human Interaction Proofs (HIPs)
   🔹 Summary:  Using Machine Learning to Break Visual Guidelines, we look for tasks where machine learning algorithms are not as good as humans with the hope of gaining insight into their current limitations .

📖 2. Online Classification on a Budget
   🔹 Summary:  In this paper we describe and analyze the simpiness of online algorithms for classification on a budget . The paper is published at the Hebrew University of Jerusalem and the University of London .

📖 3. Temporal Dynamics of Generalization in Neural Networks
   🔹 Summary:  The paper presents a rigorous characterization of how a general.nonlinear learning machine generalizes during the training process . It is trained on a random sample using a gradient .
